 **02 – DATA PREPROCESSING**

The purpose of this notebook is to clean and prepare the raw data so it’s suitable for analysis and modeling. This step includes loading the dataset, checking for issues like missing values or duplicates, and making sure the data types are correct. It also covers encoding categorical variables, scaling features, and preparing the train/test split.

**1. Load the Datasets**
   
This step loads all CSV files from the data/raw/ folder into a dictionary of pandas DataFrames, using the filenames (without .csv) as keys. The code handles potential encoding and mixed-type issues by specifying encoding (ISO-8859-1) and using dtype=str. This approach ensures easy access to and inspection of multiple datasets.

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
import os
import glob
import pandas as pd
import warnings
from pandas.errors import DtypeWarning

# Suppress DtypeWarning
warnings.simplefilter(action='ignore', category=DtypeWarning)

# Path to the raw data folder
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(raw_data_path, '*.csv'))

# Create a dictionary of DataFrames with the filename (without extension) as the key
dataframes = {}

for file in csv_files:
    name = os.path.splitext(os.path.basename(file))[0]  # filename without path or extension
    try:
        # Read the CSV with dtype=str to avoid mixed type issues
        df = pd.read_csv(file, encoding='ISO-8859-1', dtype=str, low_memory=False)
        dataframes[name] = df
        print(f"Loaded {name} with shape {df.shape}")
    except Exception as e:
        print(f"Error loading {name}: {e}")

# Optionally, check the first file to confirm
if csv_files:
    first_file = csv_files[0]
    preview_df = pd.read_csv(first_file, encoding='ISO-8859-1', dtype=str, low_memory=False)
    print(f"\nFirst file preview: {preview_df.head()}")
else:
    print("No CSV files found.")


Loaded named_anchorages_v1_20191205 with shape (166508, 10)
Loaded CVP_loitering_202411 with shape (684, 14)
Loaded named_anchorages_v1_20181108 with shape (119748, 7)
Loaded Weather-for-Boating-Activities with shape (1060, 6)
Loaded CVP_ports_202411 with shape (1410, 14)
Loaded boat_data with shape (9888, 10)
Loaded CVP_encounters_202411 with shape (348, 14)
Loaded sar_vessel_detections_pipev20231026_202410 with shape (268681, 10)
Loaded named_anchorages_v2_20201104 with shape (166515, 10)
Loaded boat_dataset with shape (10344, 38)
Loaded Boats_No_Price_dataset with shape (936, 26)
Loaded named_anchorages_v2_20221206 with shape (166482, 10)
Loaded sar_vessel_detections_pipev3_202411 with shape (248247, 10)
Loaded sar_vessel_detections_pipev3_202412 with shape (239081, 10)

First file preview:        s2id                  lat                 lon     label sublabel  \
0  3e4e429b   26.914042109356018   52.22031972443178   SHARJAH      NaN   
1  1a575de7  -7.7159917314685496   11.7245595

**2. Initial Checks: Data Types, Missing Values, and Duplicates**

This step involves checking the data types to ensure each column is correctly formatted. It also includes identifying any missing values by counting them for each column. Additionally, duplicate rows are checked for any repetitions. These checks help identify common data issues that need to be addressed before further analysis.

In [5]:
# Loop through each loaded DataFrame in the dictionary
for name, df in dataframes.items():
    print(f"\n🔍 Initial checks for: {name}")

    # Overview of the dataset
    print("\n📄 Dataset Info:")
    print(df.info())

    # Count missing values per column
    print("\n❓ Missing Values per Column:")
    print(df.isnull().sum())

    # Check for duplicates
    print("\n🔁 Number of Duplicate Rows:")
    print(df.duplicated().sum())



🔍 Initial checks for: named_anchorages_v1_20191205

📄 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166508 entries, 0 to 166507
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   s2id                   166508 non-null  object 
 1   lat                    166508 non-null  float64
 2   lon                    166508 non-null  float64
 3   label                  166502 non-null  object 
 4   sublabel               5586 non-null    object 
 5   label_source           166508 non-null  object 
 6   iso3                   166501 non-null  object 
 7   distance_from_shore_m  166483 non-null  float64
 8   drift_radius           166346 non-null  float64
 9   at_dock                166507 non-null  object 
dtypes: float64(4), object(6)
memory usage: 12.7+ MB
None

❓ Missing Values per Column:
s2id                          0
lat                           0
lon                           0


**3. Merging Anchorages Datasets**

The anchorage datasets were merged into a single DataFrame to facilitate unified analysis. This step combined multiple datasets related to anchorages, ensuring that all relevant data from different timeframes and sources is present in one dataset. This is an essential step for streamlining further data analysis and avoiding redundant information across multiple datasets.

In [6]:
# 1. Merging the Anchorages datasets
# Assuming the datasets have already been loaded into the `dataframes` dictionary

# Select the anchorage datasets
anchorage_keys = [key for key in dataframes.keys() if 'anchorages' in key]

# Print the datasets being merged
print("Merging the following Anchorage datasets:")
for key in anchorage_keys:
    print(f"- {key}")

# Merge them into a single DataFrame
df_anchorages = pd.concat([dataframes[key] for key in anchorage_keys], ignore_index=True)

# Display quick check
print("\nCombined Anchorage dataset shape:", df_anchorages.shape)
print(df_anchorages.head())


Merging the following Anchorage datasets:
- named_anchorages_v1_20191205
- named_anchorages_v1_20181108
- named_anchorages_v2_20201104
- named_anchorages_v2_20221206

Combined Anchorage dataset shape: (619253, 12)
       s2id        lat        lon     label sublabel     label_source iso3  \
0  3e4e429b  26.914042  52.220320   SHARJAH      NaN  top_destination  IRN   
1  1a575de7  -7.715992  11.724560  BLOCK 17      NaN  top_destination  AGO   
2  3fcf5295  29.642077  48.696705  KAZ IRAQ      NaN  top_destination  KWT   
3  3fcf52bf  29.644148  48.701873  KAZ IRAQ      NaN  top_destination  KWT   
4  3fcf52bd  29.639744  48.701769  UMM QASR      NaN  top_destination  KWT   

   distance_from_shore_m  drift_radius at_dock anchorage_group dock  
0                63000.0      0.056322   False             NaN  NaN  
1               134000.0      0.111111   False             NaN  NaN  
2                33000.0      0.162583   False             NaN  NaN  
3                33000.0      0.16162

**4.Handling Missing Values Across Multiple Datasets**

This step ensures that all loaded datasets are properly cleaned by addressing missing values. A loop iterates through each dataset, checking for columns with missing data. For numerical columns such as 'Length' and 'Width', missing values are filled using the median value. For categorical columns like 'Category', missing values are filled with the mode (most frequent value). If the mode is not available, missing categorical data is replaced with 'Unknown'. This approach ensures that missing data is handled consistently across all datasets, preventing errors by verifying the existence of each column before making changes. After cleaning, the missing value count is rechecked to confirm successful handling.

In [16]:
import pandas as pd
import os

# Optional: enable future behavior to avoid downcasting warning
pd.set_option('future.no_silent_downcasting', True)

# Paths
raw_data_path = '/content/drive/MyDrive/yacht-data-insights/data/raw/'
cleaned_data_path = '/content/drive/MyDrive/yacht-data-insights/data/cleaned/'

# Ensure output folder exists
os.makedirs(cleaned_data_path, exist_ok=True)

# List CSV files
csv_files = [file for file in os.listdir(raw_data_path) if file.endswith('.csv')]

# Handle each dataset
for file in csv_files:
    file_path = os.path.join(raw_data_path, file)

    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Report missing values before cleaning
        missing_before = df.isnull().sum()
        missing_before = missing_before[missing_before > 0]
        if not missing_before.empty:
            print(f"\nMissing values before cleaning in {file}:\n{missing_before}")
        else:
            print(f"\nNo missing values found in {file} before cleaning.")

        # Clean 'Length' and 'Width' if present and convert to numeric
        for col in ['Length', 'Width']:
            if col in df.columns:
                df[col] = df[col].astype(str).str.extract(r'([\d\.]+)', expand=False)
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Fill missing values
        for column in df.columns:
            if df[column].isnull().sum() > 0:
                if pd.api.types.is_numeric_dtype(df[column]):
                    median_val = df[column].median()
                    df[column] = df[column].fillna(median_val)
                    print(f"Filled missing values in numeric column '{column}' with median: {median_val}")
                else:
                    mode_val = df[column].mode().dropna()
                    if not mode_val.empty:
                        df[column] = df[column].fillna(mode_val[0])
                        print(f"Filled missing values in categorical column '{column}' with mode: {mode_val[0]}")
                    else:
                        df[column] = df[column].fillna('Unknown')
                        print(f"Filled missing values in categorical column '{column}' with 'Unknown'")

        # Ensure object types are converted properly (after fillna)
        df = df.infer_objects(copy=False)

        # Report missing values after cleaning
        missing_after = df.isnull().sum()
        missing_after = missing_after[missing_after > 0]
        if not missing_after.empty:
            print(f"\nMissing values after cleaning in {file}:\n{missing_after}")
        else:
            print(f"\nNo missing values left in {file} after cleaning.")

        # Save cleaned file
        cleaned_file_path = os.path.join(cleaned_data_path, file)
        df.to_csv(cleaned_file_path, index=False)
        print(f"Cleaned: {file}")

    except Exception as e:
        print(f"Failed to process {file}: {e}")



Missing values before cleaning in named_anchorages_v1_20191205.csv:
label                         6
sublabel                 160922
iso3                          7
distance_from_shore_m        25
drift_radius                162
at_dock                       1
dtype: int64
Filled missing values in categorical column 'label' with mode: ZHOUSHAN
Filled missing values in categorical column 'sublabel' with mode: FUJAIRAH OFFSHORE
Filled missing values in categorical column 'iso3' with mode: CHN
Filled missing values in numeric column 'distance_from_shore_m' with median: 1000.0
Filled missing values in numeric column 'drift_radius' with median: 0.09844404256206604
Filled missing values in categorical column 'at_dock' with mode: False

No missing values left in named_anchorages_v1_20191205.csv after cleaning.
Cleaned: named_anchorages_v1_20191205.csv

No missing values found in CVP_loitering_202411.csv before cleaning.

No missing values left in CVP_loitering_202411.csv after cleaning.
Clean

**5. Removing Duplicates and Cleaning Data Types**

This step removes duplicate rows to ensure the dataset contains unique entries. Afterward, it verifies that each column has the correct data type, converting numerical columns to numeric types, categorical columns to categories, and date columns to datetime format. This ensures data consistency and prepares the dataset for accurate analysis and modeling.


In [17]:
# Handle duplicates
for file in csv_files:
    file_path = os.path.join(raw_data_path, file)

    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1', low_memory=False)

        # Remove duplicates
        df = df.drop_duplicates()
        print(f"Removed duplicate rows in {file}. New row count: {len(df)}")

        # Check and convert data types after cleaning
        # Example for converting specific columns to the correct type
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Convert 'Date' column to datetime
        if 'Category' in df.columns:
            df['Category'] = df['Category'].astype('category')  # Convert 'Category' column to category type

        # Optionally, you can also convert numeric columns explicitly
        for col in ['Length', 'Width']:  # Example for numerical columns
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Save the cleaned data after removing duplicates and adjusting data types
        cleaned_file_path = os.path.join(cleaned_data_path, file)
        df.to_csv(cleaned_file_path, index=False)
        print(f"Cleaned and processed {file} (duplicates removed and data types adjusted)")

    except Exception as e:
        print(f"Failed to process {file}: {e}")


Removed duplicate rows in named_anchorages_v1_20191205.csv. New row count: 166508
Cleaned and processed named_anchorages_v1_20191205.csv (duplicates removed and data types adjusted)
Removed duplicate rows in CVP_loitering_202411.csv. New row count: 684
Cleaned and processed CVP_loitering_202411.csv (duplicates removed and data types adjusted)
Removed duplicate rows in named_anchorages_v1_20181108.csv. New row count: 119748
Cleaned and processed named_anchorages_v1_20181108.csv (duplicates removed and data types adjusted)
Removed duplicate rows in Weather-for-Boating-Activities.csv. New row count: 125
Cleaned and processed Weather-for-Boating-Activities.csv (duplicates removed and data types adjusted)
Removed duplicate rows in CVP_ports_202411.csv. New row count: 1410
Cleaned and processed CVP_ports_202411.csv (duplicates removed and data types adjusted)
Removed duplicate rows in boat_data.csv. New row count: 9888
Cleaned and processed boat_data.csv (duplicates removed and data types ad

**6. Encode Categorical Variables (Defining Independent and Dependent Variables)**

This step involves defining the independent variables (`X`) and the dependent variable (`y`) from the dataset.  
Categorical features in `X` are one-hot encoded to prepare them for use in machine learning models.  
If the target variable `y` is categorical, label encoding is applied to convert it into a numerical format.


In [1]:
# Define independent variables (X) and dependent variable (y)
X = df.drop(columns=['sold'])  # Replace 'sold' with your actual target column
y = df['sold']

# Encode categorical features in X
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

ct = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(drop='first'), categorical_cols)],
    remainder='passthrough'
)
X = ct.fit_transform(X)

# Encode the dependent variable y if it’s categorical
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

print(X.shape)
print(y[:10])


NameError: name 'df' is not defined

7. Scale Numerical Features
Numerical features were scaled using StandardScaler to ensure all features contribute equally to model performance.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Apply standard scaling
scaler = StandardScaler()
df[['numerical_column1', 'numerical_column2']] = scaler.fit_transform(df[['numerical_column1', 'numerical_column2']])

8. Split Dataset into Train/Test
The cleaned and prepared dataset was split into training and testing sets to allow model validation on unseen data.

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = df.drop('target_column', axis=1)
y = df['target_column']

# Create training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
